## Setup & Requirements
Install required libraries

A valid API key

### Libraries

In [18]:
import streamlit as st
import pandas as pd
import requests
import plotly.express as px
from datetime import datetime
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import os

In [19]:
# Base URLs
CURRENT_WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
FORECAST_URL = "https://api.openweathermap.org/data/2.5/forecast"

In [20]:
# Set your API key here
load_dotenv()  # Load variables from .env file
API_KEY = os.getenv("OPENWEATHER_API_KEY")

### Functions for getting location

In [21]:
def get_user_location():
    city = input("Enter city name: ").strip()
    state = input("Enter state code (optional): ").strip()
    country = input("Enter country code (e.g., US, GB): ").strip()
    
    # Construct the location string
    location = f"{city},{state},{country}" if state else f"{city},{country}"
    return location


### Function to retrieve current weather of given location.

In [22]:
def fetch_current_weather(location):
    url = f'{CURRENT_WEATHER_URL}?q={location}&appid={API_KEY}&units=metric'
    response = requests.get(url)
    data = response.json()
    
    if response.status_code != 200:
        print(f"Error fetching data: {data.get('message', 'Unknown error')}")
        return None
    
    #print(data)

    weather_info = {
        'City': data['name'],
        'Temperature (°C)': data['main']['temp'],
        'Humidity (%)': data['main']['humidity'],
        'Wind Speed (m/s)': data['wind']['speed'], 
        'feels_like (°C)': data['main']['feels_like'], 
        'temp_min (°C)': data['main']['temp_min'], 
        'temp_max (°C)': data['main']['temp_max'],                       
        'Description': data['weather'][0]['description'].title()
    }    

    return pd.DataFrame([weather_info])

### Function to retrieve forecast weather of given location

In [23]:
def fetch_forecast(location):
    url = f'{FORECAST_URL}?q={location}&appid={API_KEY}&units=metric'
    response = requests.get(url)
    data = response.json()

    if response.status_code != 200:
        print(f"Error fetching forecast: {data.get('message', 'Unknown error')}")
        return None

    forecast_list = data['list']
    #print(forecast_list)

    forecast_df = pd.DataFrame([{
        'DateTime': item['dt_txt'],
        'Temperature (°C)': item['main']['temp'],
        'Humidity (%)': item['main']['humidity'],
        'feels_like (°C)': item['main']['feels_like'], 
        'temp_min (°C)': item['main']['temp_min'], 
        'temp_max (°C)': item['main']['temp_max'],
        'wind (m/s)': item['wind']['speed'], 
        #'visibility': item['visibility'],
        'Weather': item['weather'][0]['description'].title()
    } for item in forecast_list])

    forecast_df['DateTime'] = pd.to_datetime(forecast_df['DateTime'])
    forecast_df['Date'] = forecast_df['DateTime'].dt.date

    return forecast_df

### Summarize forecast report Generation

In [24]:
def summarize_forecast(forecast_df):
    summary_df = forecast_df.groupby('Date').agg({
        'Temperature (°C)': ['mean', 'min', 'max'],
        'Humidity (%)': 'mean',
        'wind (m/s)': 'mean',
        'Weather': lambda x: x.mode()[0] if not x.mode().empty else None
    }).reset_index()

    summary_df.columns = ['Date', 'Avg Temp (°C)', 'Min Temp (°C)', 'Max Temp (°C)', 
                          'Avg Humidity (%)', 'Avg Wind (m/s)', 'Most Common Weather']
    return summary_df


### 📊 Visualization Function

In [25]:
def visualize_summary_1(summary_df):
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    # Temperature Plot
    summary_df.plot(x='Date', y=['Min Temp (°C)', 'Avg Temp (°C)', 'Max Temp (°C)'],
                    kind='line', marker='o', ax=ax[0])
    ax[0].set_title('Temperature Trends')
    ax[0].set_ylabel('°C')

    # Humidity and Wind
    ax2 = ax[1]
    ax2.bar(summary_df['Date'], summary_df['Avg Humidity (%)'], alpha=0.6, label='Avg Humidity (%)')
    ax2.plot(summary_df['Date'], summary_df['Avg Wind (m/s)'], color='red', marker='s', label='Avg Wind (m/s)')
    ax2.set_title('Humidity and Wind Trends')
    ax2.set_ylabel('% / m/s')
    ax2.legend()

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [26]:
def visualize_summary_2(summary_df):
    plt.figure(figsize=(12, 6))
    sns.set(style="whitegrid")

    # Plot Temperature
    plt.subplot(1, 2, 1)
    sns.lineplot(x='Date', y='Avg Temp (°C)', data=summary_df, marker='o', color='tomato')
    plt.title('Average Temperature (°C) Over Days')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Plot Humidity
    plt.subplot(1, 2, 2)
    sns.lineplot(x='Date', y='Avg Humidity (%)', data=summary_df, marker='o', color='dodgerblue')
    plt.title('Average Humidity (%) Over Days')
    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.suptitle("5-Day Weather Forecast Summary", fontsize=16, fontweight='bold', y=1.05)
    plt.show()


In [27]:
import plotly.express as px

def interactive_visualize_summary(summary_df):
    fig = px.line(summary_df, x='Date', y=['Avg Temp (°C)', 'Avg Humidity (%)'],
                  title='Interactive Daily Weather Summary',
                  markers=True)
    fig.update_layout(xaxis_title='Date', yaxis_title='Value')
    fig.show()

### Display Summary

In [28]:
def display_summary_styled(summary_df):
    styled_df = summary_df.style.background_gradient(subset=['Avg Temp (°C)'], cmap='coolwarm') \
                                 .background_gradient(subset=['Avg Humidity (%)'], cmap='Blues') \
                                 .set_caption("📊 Daily Weather Summary")
    display(styled_df)  # Use display() if in Jupyter, else styled_df.to_html() for web
    visualize_summary_1(summary_df)
    visualize_summary_2(summary_df)
    interactive_visualize_summary(summary_df)
    



In [29]:
def display_summary(summary_df):
    print("\n📊 Daily Weather Summary:")
    print(tabulate(summary_df, headers='keys', tablefmt='fancy_grid', showindex=False))  
    

### Main Function

In [30]:
def main():
    location = get_user_location()

    print("\n📍 Fetching current weather...")
    current_weather = fetch_current_weather(location)
    if current_weather is not None:
        print(current_weather)

    print("\n📅 Fetching 5-day forecast...")
    forecast = fetch_forecast(location)
    if forecast is not None:
        print("\n🔍 Forecast Preview:")
        print(forecast.head())

        print("\n📊 Daily Summary:")
        summary = summarize_forecast(forecast)
        display_summary_styled(summary)
        # display_summary(summary)
      
# Run the program
if __name__ == "__main__":
    main()



📍 Fetching current weather...
      City  Temperature (°C)  Humidity (%)  Wind Speed (m/s)  feels_like (°C)  \
0  Chennai             31.37            65              5.66            37.06   

   temp_min (°C)  temp_max (°C) Description  
0          30.61           31.7  Few Clouds  

📅 Fetching 5-day forecast...

🔍 Forecast Preview:
             DateTime  Temperature (°C)  Humidity (%)  feels_like (°C)  \
0 2025-06-02 03:00:00             30.97            65            36.08   
1 2025-06-02 06:00:00             32.38            56            36.74   
2 2025-06-02 09:00:00             33.44            52            37.83   
3 2025-06-02 12:00:00             30.81            64            35.44   
4 2025-06-02 15:00:00             30.38            71            36.25   

   temp_min (°C)  temp_max (°C)  wind (m/s)           Weather        Date  
0          30.97          31.92        5.50        Few Clouds  2025-06-02  
1          32.38          35.20        4.84  Scattered Clouds  202

,Date,Avg Temp (°C),Min Temp (°C),Max Temp (°C),Avg Humidity (%),Avg Wind (m/s),Most Common Weather
0,2025-06-02,31.220000,30.190000,33.440000,62.714286,5.432857,Light Rain
1,2025-06-03,32.000000,30.080000,35.460000,54.250000,5.796250,Overcast Clouds
2,2025-06-04,31.595000,29.430000,34.900000,54.125000,5.485000,Overcast Clouds
3,2025-06-05,31.216250,28.810000,34.510000,60.000000,5.058750,Overcast Clouds
4,2025-06-06,31.125000,28.900000,34.380000,66.750000,5.741250,Overcast Clouds
5,2025-06-07,29.320000,29.320000,29.320000,76.000000,6.200000,Broken Clouds


C:\Users\sahil\AppData\Local\Temp\ipykernel_25584\1063960050.py:21: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

C:\Users\sahil\AppData\Local\Temp\ipykernel_25584\1816952130.py:20: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

